# Integrating a Custom LLM with SigLLM

This tutorial shows how to use your own LLM backend (like Azure OpenAI, Anthropic, or an internal API) with SigLLM's forecasting pipeline.

## Overview

SigLLM provides a `CustomForecast` primitive that works with any LLM. You just need to:
1. Implement a client class with a `generate()` method
2. Pass it to the `custom_detector` pipeline


## Step 1: Implement Your LLM Client

Create a class that inherits from `BaseLLMClient`. Here's an example for Azure OpenAI:


In [ ]:
from sigllm.primitives.llm_client import BaseLLMClient
from openai import AzureOpenAI

class AzureGPTClient(BaseLLMClient):
    """Azure OpenAI client for SigLLM."""
    
    def __init__(self, endpoint, api_key, deployment, api_version="2024-02-15-preview"):
        self.endpoint = endpoint
        self.api_key = api_key
        self.deployment = deployment
        self.api_version = api_version
    
    def generate(self, prompts, system_message=None, max_tokens=100, 
                 temperature=1.0, n_samples=1, **kwargs):
        """Generate responses for a batch of prompts."""
        all_responses = []
        for prompt in prompts:
            messages = []
            if system_message:
                messages.append({"role": "system", "content": system_message})
            messages.append({"role": "user", "content": prompt})
            
            response = self.client.chat.completions.create(
                model=self.deployment,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
                n=n_samples,
            )
            texts = [choice.message.content for choice in response.choices]
            all_responses.append(texts)
        
        return all_responses


## Step 2: Create Test Data


In [2]:
import numpy as np
import pandas as pd

# Simple linear test data
np.random.seed(42)
n_points = 50
data = pd.DataFrame({
    "timestamp": np.arange(n_points) * 3600,
    "value": np.linspace(0, n_points-1, n_points) * 10,
})

print("Data shape:", data.shape)
data.head()


Data shape: (50, 2)


,timestamp,value
0,0,0.0
1,3600,10.0
2,7200,20.0
3,10800,30.0
4,14400,40.0


## Step 3: Run the Pipeline


In [10]:
from mlblocks import MLPipeline

client = AzureGPTClient(
    endpoint=AZURE_ENDPOINT,
    api_key=AZURE_API_KEY,
    deployment=AZURE_DEPLOYMENT,
)

pipeline = MLPipeline(
    "custom_detector",
    init_params={
        "sigllm.primitives.forecasting.custom.CustomForecast#1": {
            "client": client,
            "steps": 1,
            "temp": 0.3,
            "samples": 4,
        },
        "mlstars.custom.timeseries_preprocessing.time_segments_aggregate#1": {
            "time_column": "timestamp",
            "interval": 3600,
        },
        "sigllm.primitives.forecasting.custom.rolling_window_sequences#1": {
            "window_size": 10,
            "target_column": 0,
            "target_size": 1,
        },
        "sigllm.primitives.transformation.format_as_integer#1": {
            "trunc": 1,
            "errors": "coerce"
        }
    }
)

print("Pipeline primitives:")
for i, p in enumerate(pipeline.primitives):
    print(f"\t{i}: {p}")


Pipeline primitives:
	0: mlstars.custom.timeseries_preprocessing.time_segments_aggregate
	1: sklearn.impute.SimpleImputer
	2: sigllm.primitives.transformation.Float2Scalar
	3: sigllm.primitives.forecasting.custom.rolling_window_sequences
	4: sigllm.primitives.transformation.format_as_string
	5: sigllm.primitives.forecasting.custom.CustomForecast
	6: sigllm.primitives.transformation.format_as_integer
	7: sigllm.primitives.transformation.Scalar2Float
	8: sigllm.primitives.postprocessing.aggregate_rolling_window
	9: numpy.reshape
	10: orion.primitives.timeseries_errors.regression_errors
	11: orion.primitives.timeseries_anomalies.find_anomalies


In [11]:
context = pipeline.fit(data, output_=7)

y_hat = context.get('y_hat')
y = context.get('y')

print(f"Predictions shape: {y_hat.shape}")
print(f"Actuals shape: {y.shape}")

for i in range(min(5, len(y_hat))):
    pred = float(np.array(y_hat[i]).flatten()[0])
    actual = float(np.array(y[i]).flatten()[0])
    print(f"Window {i}: pred={pred:.1f}, actual={actual:.1f}, error={pred-actual:.1f}")


Predictions shape: (40, 4, 1)
Actuals shape: (40, 1)
Window 0: pred=100.0, actual=100.0, error=0.0
Window 1: pred=110.0, actual=110.0, error=0.0
Window 2: pred=120.0, actual=120.0, error=0.0
Window 3: pred=130.0, actual=130.0, error=0.0
Window 4: pred=140.0, actual=140.0, error=0.0


## The `generate()` Interface

Your `generate()` method must accept these parameters:

| Parameter | Type | Description |
|-----------|------|-------------|
| `prompts` | `List[str]` | List of prompt strings |
| `system_message` | `str` or `None` | Optional system context |
| `max_tokens` | `int` | Max tokens per response |
| `temperature` | `float` | Sampling temperature |
| `n_samples` | `int` | Responses per prompt |
| `**kwargs` | | Additional parameters |

**Returns:** `List[List[str]]` — for each prompt, a list of `n_samples` response strings.
